In [51]:
import pandas as pd

In [62]:

import pandas as pd
import logging
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Configure logging
logging.basicConfig(level=logging.INFO)

movies = pd.read_csv(r"C:\Users\T\Documents\Python Training\Python Class\Projects\Movie Recommendation\tmdb_5000_movies.csv")
# Print as WARNING
logging.warning("Movies Data loaded successfully!")
#print(movies.head(2))

credits = pd.read_csv(r"C:\Users\T\Documents\Python Training\Python Class\Projects\Movie Recommendation\tmdb_5000_credits.csv")
# Print as WARNING
logging.warning("Credit Data loaded successfully!")
#print(credits.head(2))

# Merge the two DataFrames on the 'title' column
movies = movies.merge(credits, on="title")

# Select relevant columns
movies = movies[["id","title", "overview","genres", "keywords", "cast", "crew"]]
# movies.head(1)


# Using ast.literal_eval to parse a string representation of a list/dictionary
#ast.literal_eval(movies["genres"][0])

#ast.literal_eval(movies["crew"][0])


#creating functio for literal
#ast.literal_eval library function is used to parse the dictionary into individual section
#function to convert stringified list of dictionaries to list of names
def convert(text):
    l = []
    for i in ast.literal_eval(text): #ast.literal_eval(movies["crew"][0])
        l.append(i["name"])
    return l

#checking for null values
#movies.isna().sum()

# removing null values
movies = movies.dropna()


#Applying Convert function to pasrse data
movies["genres"] = movies["genres"].apply(convert)

#Applying Convert function to pasrse data
movies["keywords"] = movies["keywords"].apply(convert)


# limiting cast to first 5 name

#creating functio for literal
#ast.literal_eval library function is used to parse the dictionary into individual section
def convert_1(text):
    l = []
    count = 0
    for i in ast.literal_eval(text): #ast.literal_eval(movies["crew"][0])
        if count < 5:
            l.append(i["name"])
        count += 1
    return l

#Applying Convert_1 limit 5 element function to pasrse data
movies["cast"] = movies["cast"].apply(convert_1)



# parsing director name on the basis of their job

#creating functio for literal
#ast.literal_eval library function is used to parse the dictionary into individual section
def director(text):
    l = []
    for i in ast.literal_eval(text): #ast.literal_eval(movies["crew"][0])
        if i["job"]=="Director":
            l.append(i["name"])
    return l

#Applying director function to crew
movies["crew"] = movies["crew"].apply(director)

# function to remove extra space from the text
def rem(li):
    l=[]
    for i in li:
        l.append(i.replace(" ",""))
    return l

# applying cleanup function to all columns

movies["genres"] = movies["genres"].apply(rem)
movies["keywords"] = movies["keywords"].apply(rem)
movies["cast"] = movies["cast"].apply(rem)
movies["crew"] = movies["crew"].apply(rem)

#splitting the overview column data into a list
# all the other columns were allready converted to list

movies["overview"] = movies["overview"].apply(lambda x: x.split())

# creating a new column 'tags' by combining all the relevant columns
# creating a has like values in single line
movies["tags"] = movies["overview"]+movies["genres"]+movies["keywords"]+movies["cast"]+movies["crew"]

#converting the tag list into a single sentence
movies["tags"]= movies["tags"].apply(lambda x: " ".join(x));
#movies["tags"][0]

# creating a new data frame for processing
new = movies[["id","title","tags"]]

# converting all the tags into lower case
new.loc[:, "tags"] = new["tags"].apply(lambda x: x.lower())

#cleaning up stopword
cv = CountVectorizer(stop_words="english")


#for visualization converting it into array
vector = cv.fit_transform(new["tags"]).toarray()


similarity = cosine_similarity(vector)

# Recommendation function (fixed and improved)
def recommended_movie(movie_title, n_recommendations=5):
    try:
        # Find the index of the input movie
        index = new[new['title'] == movie_title].index[0]
        
        # Get similarity scores and sort in descending order
        distances = sorted(
            list(enumerate(similarity[index])), 
            reverse=True, 
            key=lambda x: x[1]
        )
        
        print(f"\nTop {n_recommendations} movies similar to '{movie_title}':\n")
        for i in distances[1:n_recommendations + 1]:  # Skip the movie itself
            print(new.iloc[i[0]].title)
            
    except IndexError:
        print(f"Movie '{movie_title}' not found in the database.")


In [63]:
#movies.isna().sum()
#movies.head(1)

#movies["genres"][0]
#movies["keywords"][0]
#movies["cast"][0]
#movies["overview"][0]
#movies["tags"][0]
#new.head(5)
#vector[0]

# Example: Get index of a movie
#print(new[new['title'] == 'Newlyweds'].index[0])
# Test the function
recommended_movie('Newlyweds')


Top 5 movies similar to 'Newlyweds':

Firestarter
Captive
Elizabethtown
Bad Grandpa
How to Fall in Love
